# Broadcasting Fundamentals

**Module 04 | Notebook 01**

---

## Objective
By the end of this notebook, you will master:
- What broadcasting is and why it matters
- Broadcasting rules
- Common broadcasting patterns
- Debugging broadcasting errors
- Memory efficiency with broadcasting

In [2]:
import numpy as np
np.set_printoptions(precision=2)

---
## 1. What is Broadcasting?

Broadcasting allows NumPy to perform operations on arrays of **different shapes** without explicitly copying data. It "stretches" smaller arrays to match larger ones.

In [3]:
# Simple example: scalar + array
arr = np.array([1, 2, 3, 4, 5])
result = arr + 10

print(f"arr: {arr}")
print(f"arr + 10: {result}")

# The scalar 10 is "broadcast" to match arr's shape

arr: [1 2 3 4 5]
arr + 10: [11 12 13 14 15]


In [4]:
# Without broadcasting (inefficient)
arr = np.array([1, 2, 3, 4, 5])
scalar_expanded = np.full(arr.shape, 10)  # Create array of 10s

print(f"Manual expansion: {scalar_expanded}")
print(f"Result: {arr + scalar_expanded}")

Manual expansion: [10 10 10 10 10]
Result: [11 12 13 14 15]


In [5]:
# 2D example
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
row = np.array([10, 20, 30])

print(f"Matrix (3x3):\n{matrix}")
print(f"Row (3,): {row}")
print(f"Matrix + Row:\n{matrix + row}")

Matrix (3x3):
[[1 2 3]
 [4 5 6]
 [7 8 9]]
Row (3,): [10 20 30]
Matrix + Row:
[[11 22 33]
 [14 25 36]
 [17 28 39]]


---
## 2. Broadcasting Rules

Two arrays are compatible for broadcasting if for each dimension (trailing):
1. The dimensions are equal, OR
2. One of the dimensions is 1, OR
3. One of the arrays has fewer dimensions

**Dimensions are compared from right to left (trailing dimensions first)**

In [6]:
# Rule visualization
print("Shape Compatibility Examples:")
print("(3,)   + scalar -> Works: scalar broadcasts to (3,)")
print("(3, 4) + (4,)   -> Works: (4,) broadcasts to (3, 4)")
print("(3, 4) + (3, 1) -> Works: (3, 1) broadcasts to (3, 4)")
print("(3, 4) + (3,)   -> FAILS: 4 != 3")

Shape Compatibility Examples:
(3,)   + scalar -> Works: scalar broadcasts to (3,)
(3, 4) + (4,)   -> Works: (4,) broadcasts to (3, 4)
(3, 4) + (3, 1) -> Works: (3, 1) broadcasts to (3, 4)
(3, 4) + (3,)   -> FAILS: 4 != 3


In [7]:
# Example 1: (3, 4) + (4,)
A = np.ones((3, 4))
B = np.array([1, 2, 3, 4])

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"Result shape: {(A + B).shape}")
print(f"Result:\n{A + B}")

A shape: (3, 4)
B shape: (4,)
Result shape: (3, 4)
Result:
[[2. 3. 4. 5.]
 [2. 3. 4. 5.]
 [2. 3. 4. 5.]]


In [8]:
# Example 2: (3, 4) + (3, 1)
A = np.ones((3, 4))
B = np.array([[10], [20], [30]])  # Shape (3, 1)

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"Result:\n{A + B}")

A shape: (3, 4)
B shape: (3, 1)
Result:
[[11. 11. 11. 11.]
 [21. 21. 21. 21.]
 [31. 31. 31. 31.]]


In [9]:
# Example 3: (3, 4) + (3,) - FAILS!
A = np.ones((3, 4))
B = np.array([10, 20, 30])  # Shape (3,)

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")

try:
    result = A + B
except ValueError as e:
    print(f"Error: {e}")

A shape: (3, 4)
B shape: (3,)
Error: operands could not be broadcast together with shapes (3,4) (3,) 


In [10]:
# Fix: Reshape B to (3, 1)
B_reshaped = B.reshape(3, 1)  # or B[:, np.newaxis]
print(f"B reshaped: {B_reshaped.shape}")
print(f"Result:\n{A + B_reshaped}")

B reshaped: (3, 1)
Result:
[[11. 11. 11. 11.]
 [21. 21. 21. 21.]
 [31. 31. 31. 31.]]


---
## 3. Adding Dimensions with np.newaxis

In [11]:
arr = np.array([1, 2, 3])  # Shape (3,)
print(f"Original shape: {arr.shape}")

Original shape: (3,)


In [12]:
# Add axis at position 0 -> row vector (1, 3)
row = arr[np.newaxis, :]
print(f"Row vector: {row.shape}")
print(row)

Row vector: (1, 3)
[[1 2 3]]


In [13]:
# Add axis at position 1 -> column vector (3, 1)
col = arr[:, np.newaxis]
print(f"Column vector: {col.shape}")
print(col)

Column vector: (3, 1)
[[1]
 [2]
 [3]]


In [14]:
# Alternative: None is alias for np.newaxis
print(f"Using None: {arr[None, :].shape}")
print(f"Using None: {arr[:, None].shape}")

Using None: (1, 3)
Using None: (3, 1)


In [15]:
# Practical: Outer product using broadcasting
a = np.array([1, 2, 3])
b = np.array([10, 20])

# a as column (3, 1), b as row (1, 2) -> result (3, 2)
outer = a[:, np.newaxis] * b[np.newaxis, :]
print(f"Outer product:\n{outer}")

# Verify with np.outer
print(f"np.outer:\n{np.outer(a, b)}")

Outer product:
[[10 20]
 [20 40]
 [30 60]]
np.outer:
[[10 20]
 [20 40]
 [30 60]]


---
## 4. Common Broadcasting Patterns

In [16]:
# Pattern 1: Row-wise operation
# Subtract row mean from each row
data = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
row_means = data.mean(axis=1, keepdims=True)  # Shape (3, 1)

print(f"Data:\n{data}")
print(f"Row means: {row_means.flatten()}")
print(f"Centered:\n{data - row_means}")

Data:
[[1 2 3]
 [4 5 6]
 [7 8 9]]
Row means: [2. 5. 8.]
Centered:
[[-1.  0.  1.]
 [-1.  0.  1.]
 [-1.  0.  1.]]


In [17]:
# Pattern 2: Column-wise operation
# Divide each column by its max
data = np.array([[1, 4, 7], [2, 5, 8], [3, 6, 9]], dtype=float)
col_max = data.max(axis=0, keepdims=True)  # Shape (1, 3)

print(f"Data:\n{data}")
print(f"Column max: {col_max}")
print(f"Normalized:\n{data / col_max}")

Data:
[[1. 4. 7.]
 [2. 5. 8.]
 [3. 6. 9.]]
Column max: [[3. 6. 9.]]
Normalized:
[[0.33 0.67 0.78]
 [0.67 0.83 0.89]
 [1.   1.   1.  ]]


In [18]:
# Pattern 3: Add bias to each sample (batch processing)
batch = np.random.rand(100, 10)  # 100 samples, 10 features
bias = np.random.rand(10)        # Bias for each feature

# bias (10,) broadcasts to (100, 10)
batch_with_bias = batch + bias
print(f"Batch shape: {batch.shape}")
print(f"Bias shape: {bias.shape}")
print(f"Result shape: {batch_with_bias.shape}")

Batch shape: (100, 10)
Bias shape: (10,)
Result shape: (100, 10)


In [19]:
# Pattern 4: Distance matrix (pairwise distances)
# Points in 2D
points = np.array([[0, 0], [1, 0], [0, 1], [1, 1]])
n = len(points)

# Reshape for broadcasting: (n, 1, 2) - (1, n, 2) = (n, n, 2)
diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
distances = np.sqrt((diff ** 2).sum(axis=2))

print(f"Points:\n{points}")
print(f"Distance matrix:\n{distances}")

Points:
[[0 0]
 [1 0]
 [0 1]
 [1 1]]
Distance matrix:
[[0.   1.   1.   1.41]
 [1.   0.   1.41 1.  ]
 [1.   1.41 0.   1.  ]
 [1.41 1.   1.   0.  ]]


---
## 5. np.broadcast_to and np.broadcast_arrays

In [20]:
# broadcast_to: explicitly broadcast to shape
arr = np.array([1, 2, 3])

broadcasted = np.broadcast_to(arr, (4, 3))
print(f"Original shape: {arr.shape}")
print(f"Broadcasted shape: {broadcasted.shape}")
print(f"Broadcasted:\n{broadcasted}")

Original shape: (3,)
Broadcasted shape: (4, 3)
Broadcasted:
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [1 2 3]]


In [21]:
# broadcast_to creates a VIEW (no copy!)
print(f"Original nbytes: {arr.nbytes}")
print(f"Broadcasted nbytes: {broadcasted.nbytes}")
# Same memory! The 'repetition' is virtual via strides

Original nbytes: 24
Broadcasted nbytes: 96


In [22]:
# broadcast_arrays: broadcast multiple arrays together
a = np.array([1, 2, 3])
b = np.array([[10], [20]])

a_bc, b_bc = np.broadcast_arrays(a, b)
print(f"a original: {a.shape} -> broadcasted: {a_bc.shape}")
print(f"b original: {b.shape} -> broadcasted: {b_bc.shape}")
print(f"a broadcasted:\n{a_bc}")
print(f"b broadcasted:\n{b_bc}")

a original: (3,) -> broadcasted: (2, 3)
b original: (2, 1) -> broadcasted: (2, 3)
a broadcasted:
[[1 2 3]
 [1 2 3]]
b broadcasted:
[[10 10 10]
 [20 20 20]]


In [23]:
# np.broadcast_shapes: determine result shape without computing
shape1 = (3, 1, 4)
shape2 = (5, 4)

result_shape = np.broadcast_shapes(shape1, shape2)
print(f"Broadcasting {shape1} + {shape2} -> {result_shape}")

Broadcasting (3, 1, 4) + (5, 4) -> (3, 5, 4)


---
## 6. Memory Efficiency

In [24]:
# Compare: tile vs broadcast_to
arr = np.arange(1000)

# tile creates actual copy
tiled = np.tile(arr, (1000, 1))

# broadcast_to uses strides (virtual)
broadcasted = np.broadcast_to(arr, (1000, 1000))

print(f"Original: {arr.nbytes / 1024:.1f} KB")
print(f"Tiled: {tiled.nbytes / 1024:.1f} KB")
print(f"Broadcasted: {broadcasted.nbytes / 1024:.1f} KB")

# Note: broadcasted reports same memory as underlying array

Original: 7.8 KB
Tiled: 7812.5 KB
Broadcasted: 7812.5 KB


In [25]:
# Strides reveal the trick
print(f"Original strides: {arr.strides}")
print(f"Broadcasted strides: {broadcasted.strides}")
# Stride of 0 means no movement in memory!

Original strides: (8,)
Broadcasted strides: (0, 8)


---
## 7. Debugging Broadcasting Errors

In [26]:
# Typical error
A = np.ones((3, 4, 5))
B = np.ones((4, 6))

try:
    result = A + B
except ValueError as e:
    print(f"Error: {e}")
    print(f"\nA shape: {A.shape}")
    print(f"B shape: {B.shape}")
    print("\nCompare from right:")
    print("  A: (3, 4, 5)")
    print("  B:    (4, 6)")
    print("         ^  ^ ")
    print("  5 != 6 (FAIL)")

Error: operands could not be broadcast together with shapes (3,4,5) (4,6) 

A shape: (3, 4, 5)
B shape: (4, 6)

Compare from right:
  A: (3, 4, 5)
  B:    (4, 6)
         ^  ^ 
  5 != 6 (FAIL)


In [27]:
# Helper function to check compatibility
def can_broadcast(shape1, shape2):
    """Check if two shapes are broadcast compatible."""
    try:
        np.broadcast_shapes(shape1, shape2)
        return True
    except ValueError:
        return False

print(f"(3, 4) + (4,): {can_broadcast((3, 4), (4,))}")
print(f"(3, 4) + (3,): {can_broadcast((3, 4), (3,))}")
print(f"(3, 4) + (3, 1): {can_broadcast((3, 4), (3, 1))}")

(3, 4) + (4,): True
(3, 4) + (3,): False
(3, 4) + (3, 1): True


---
## Key Points Summary

**Broadcasting Rules:**
1. Arrays with fewer dimensions get 1s prepended to shape
2. Size 1 in any dimension broadcasts to match
3. Dimensions compared right-to-left
4. Must have same size OR one must be 1

**Key Functions:**
- `np.newaxis` (or `None`): Add dimension
- `np.broadcast_to()`: Virtual broadcast (view)
- `np.broadcast_arrays()`: Broadcast multiple arrays
- `np.broadcast_shapes()`: Get result shape

**Memory:**
- Broadcasting is virtual (no copy)
- Uses stride tricks
- Much more efficient than `tile` or manual copying

---
## Interview Tips

**Q1: What is broadcasting in NumPy?**
> Broadcasting allows operations on arrays of different shapes by virtually replicating smaller arrays. No actual data copying occurs - NumPy uses stride tricks.

**Q2: Given shapes (3, 4, 5) and (4, 1), what is the result shape?**
> Compare right-to-left:
> - (3, 4, 5) vs (4, 1)
> - 5 vs 1: OK (broadcast 1 to 5)
> - 4 vs 4: OK (same)
> - 3 vs missing: OK (prepend 1)
> Result: (3, 4, 5)

**Q3: How do you add a column vector to a matrix?**
> Reshape vector to (n, 1) using `vec[:, np.newaxis]` or `vec.reshape(-1, 1)`, then add.

**Q4: Why is broadcasting memory efficient?**
> It uses stride of 0 for broadcast dimensions, so the same memory location is accessed repeatedly without copying data.

---
## Practice Exercises

### Exercise 1: Standardize each column (Z-score normalization)

In [28]:
# Standardize each column to mean=0, std=1
data = np.array([[1, 100, 50], [2, 200, 60], [3, 300, 70], [4, 400, 80]])


In [29]:
# Solution
data = np.array([[1, 100, 50], [2, 200, 60], [3, 300, 70], [4, 400, 80]], dtype=float)

mean = data.mean(axis=0, keepdims=True)
std = data.std(axis=0, keepdims=True)

standardized = (data - mean) / std
print(f"Original:\n{data}")
print(f"Standardized:\n{standardized}")
print(f"Verify mean: {standardized.mean(axis=0)}")
print(f"Verify std: {standardized.std(axis=0)}")

Original:
[[  1. 100.  50.]
 [  2. 200.  60.]
 [  3. 300.  70.]
 [  4. 400.  80.]]
Standardized:
[[-1.34 -1.34 -1.34]
 [-0.45 -0.45 -0.45]
 [ 0.45  0.45  0.45]
 [ 1.34  1.34  1.34]]
Verify mean: [0. 0. 0.]
Verify std: [1. 1. 1.]


### Exercise 2: Create multiplication table using broadcasting

In [30]:
# Create 10x10 multiplication table


In [31]:
# Solution
nums = np.arange(1, 11)

# Use broadcasting: (10, 1) * (1, 10) = (10, 10)
mult_table = nums[:, np.newaxis] * nums[np.newaxis, :]
print(f"Multiplication Table:\n{mult_table}")

Multiplication Table:
[[  1   2   3   4   5   6   7   8   9  10]
 [  2   4   6   8  10  12  14  16  18  20]
 [  3   6   9  12  15  18  21  24  27  30]
 [  4   8  12  16  20  24  28  32  36  40]
 [  5  10  15  20  25  30  35  40  45  50]
 [  6  12  18  24  30  36  42  48  54  60]
 [  7  14  21  28  35  42  49  56  63  70]
 [  8  16  24  32  40  48  56  64  72  80]
 [  9  18  27  36  45  54  63  72  81  90]
 [ 10  20  30  40  50  60  70  80  90 100]]


### Exercise 3: Apply different weights to each sample

In [32]:
# Apply sample-specific weights (each row gets different weight)
samples = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
weights = np.array([0.5, 1.0, 2.0])  # Weight for each row


In [33]:
# Solution
samples = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=float)
weights = np.array([0.5, 1.0, 2.0])

# Reshape weights to (3, 1) to broadcast across columns
weighted = samples * weights[:, np.newaxis]
print(f"Original:\n{samples}")
print(f"Weights: {weights}")
print(f"Weighted:\n{weighted}")

Original:
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Weights: [0.5 1.  2. ]
Weighted:
[[ 0.5  1.   1.5]
 [ 4.   5.   6. ]
 [14.  16.  18. ]]


---
## Next Notebook
**02_vectorization_techniques.ipynb** - Replace loops with vectorized operations for maximum performance.